In [1]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, unix_timestamp, hour, weekofyear, month, quarter, dayofweek, dayofmonth, dayofyear, to_date
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler as sklearn_min_max_scaler
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,  StringIndexer
#import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [4]:
spark = SparkSession.builder.config("spark.driver.memory", "4g").config("spark.executor.memory", "4g").appName("predictconidtion").getOrCreate()


your 131072x1 screen size is bogus. expect trouble
24/12/10 16:13:43 WARN Utils: Your hostname, LAPTOP-DN3PCHGN resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/10 16:13:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/10 16:13:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Đọc dữ liệu từ file parquet
parquet_path = "/home/hoan123/project-master/delta_lake/delta_lake_table_3/"
df = spark.read.parquet(parquet_path)

In [6]:
# Sắp xếp dữ liệu theo cột datetime
df_sorted = df.orderBy("datetime")


In [7]:
condition_to_index = {
    "Cloudy": 0,
    "Light rain": 1,
    "Patchy rain possible": 2,
    "Moderate or heavy rain shower": 3,
    "Patchy light rain": 4,
    "Clear": 5,
    "Partly cloudy": 6,
    "Sunny": 7,
    "Patchy light drizzle": 8,
    "Light drizzle": 9,
    "Patchy light rain with thunder": 10,
    "Mist": 11,
    "Moderate rain at times": 12,
    "Thundery outbreaks possible": 13,
    "Moderate or heavy rain with thunder": 14,
    "Overcast": 15,
    "Moderate rain": 16,
    "Light rain shower": 17,
    "Heavy rain": 18,
    "Heavy rain at times": 19,
    "Fog": 20
}

# Bước 2: Hàm chuyển đổi chuỗi sang chỉ số
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def map_condition(condition):
    return condition_to_index.get(condition, -1)  # Trả về -1 nếu không tìm thấy giá trị phù hợp

map_condition_udf = udf(map_condition, IntegerType())

# Bước 3: Áp dụng UDF để tạo các cột condition đã mã hóa
df_sorted = df_sorted.withColumn("condition_index", map_condition_udf("condition"))

# Hiển thị kết quả
df_sorted.show()


+-------------------+-----+-------+--------+---------+------+--------+--------+---------+--------------------+---------------+
|           datetime| name|country|latitude|longitude|temp_c|wind_mph|humidity|precip_mm|           condition|condition_index|
+-------------------+-----+-------+--------+---------+------+--------+--------+---------+--------------------+---------------+
|2024-01-01 00:00:00|Hanoi|Vietnam| 21.0333|   105.85|  20.7|     5.8|    82.0|      0.0|       Partly cloudy|              6|
|2024-01-01 01:00:00|Hanoi|Vietnam| 21.0333|   105.85|  20.4|     6.0|    84.0|      0.0|              Cloudy|              0|
|2024-01-01 02:00:00|Hanoi|Vietnam| 21.0333|   105.85|  20.0|     4.9|    85.0|      0.0|              Cloudy|              0|
|2024-01-01 03:00:00|Hanoi|Vietnam| 21.0333|   105.85|  19.8|     4.5|    86.0|      0.0|            Overcast|             15|
|2024-01-01 04:00:00|Hanoi|Vietnam| 21.0333|   105.85|  19.6|     4.7|    87.0|      0.0|            Overcast| 

In [12]:

window = Window.orderBy("datetime")

# Thêm các cột feature từ các bước thời gian trước
df_with_lags = (
    df_sorted
    .withColumn("temp_c_t-1", lag("temp_c", 1).over(window))
    .withColumn("temp_c_t-2", lag("temp_c", 2).over(window))
    .withColumn("temp_c_t-3", lag("temp_c", 3).over(window))
    .withColumn("temp_c_t-4", lag("temp_c", 4).over(window))
    .withColumn("humidity_t-1", lag("humidity", 1).over(window))
    .withColumn("humidity_t-2", lag("humidity", 2).over(window))
    .withColumn("humidity_t-3", lag("humidity", 3).over(window))
    .withColumn("humidity_t-4", lag("humidity", 4).over (window))
    .withColumn("wind_mph_t-1", lag("wind_mph", 1).over(window))
    .withColumn("wind_mph_t-2", lag("wind_mph", 2).over(window))
    .withColumn("wind_mph_t-3", lag("wind_mph", 3).over(window))
    .withColumn("wind_mph_t-4", lag("wind_mph", 4).over(window))
    .withColumn("precip_mm_t-1", lag("precip_mm", 1).over(window))
    .withColumn("precip_mm_t-2", lag("precip_mm", 2).over(window))
    .withColumn("precip_mm_t-3", lag("precip_mm", 3).over(window))
    .withColumn("precip_mm_t-4", lag("precip_mm", 4).over(window))
    .withColumn("condition_t-1", lag("condition_index", 1).over(window))
    .withColumn("condition_t-2", lag("condition_index", 2).over(window))
    .withColumn("condition_t-3", lag("condition_index", 3).over(window))
    .withColumn("condition_t-4", lag("condition_index", 4).over(window))
    .withColumn("hour", hour("datetime"))
    .withColumn("day_of_month", dayofmonth("datetime"))
    .withColumn("day_of_week", dayofweek("datetime"))
    .withColumn("month", month("datetime"))
    .withColumn("quarter", quarter("datetime"))
    .withColumn("week", weekofyear("datetime"))
)

# Chọn các cột cần thiết và đổi tên cột label
df_final = df_with_lags.select(
    "datetime", 
    "temp_c_t-1", "temp_c_t-2", "temp_c_t-3", "temp_c_t-4", 
    "humidity_t-1", "humidity_t-2", "humidity_t-3", "humidity_t-4",
    "wind_mph_t-1", "wind_mph_t-2","wind_mph_t-3","wind_mph_t-4",
    "precip_mm_t-1", "precip_mm_t-2","precip_mm_t-3","precip_mm_t-4",
    "condition_t-1", "condition_t-2", "condition_t-3", "condition_t-4",  
    "hour", "day_of_month", "day_of_week", "month", "quarter", "week", 
    "condition_index"  # Nhiệt độ hiện tại làm nhãn
).withColumnRenamed("condition_index", "label")

# Bỏ các hàng không đủ dữ liệu
df_final = df_final.na.drop()


In [13]:


# Thêm cột 'date' để lọc train-test
df_final = df_final.withColumn("date", to_date("datetime"))

# Chia dữ liệu train-test
train_data = df_final.filter(col("date") < "2024-10-30").drop("date")
test_data = df_final.filter(col("date") >= "2024-10-30").drop("date")

# Tạo feature vector
feature_cols = [
    "temp_c_t-1", "temp_c_t-2", "temp_c_t-3", "temp_c_t-4", 
    "humidity_t-1", "humidity_t-2",  "humidity_t-3", "humidity_t-4",
    "wind_mph_t-1", "wind_mph_t-2", "wind_mph_t-3","wind_mph_t-4",
    "precip_mm_t-1", "precip_mm_t-2","precip_mm_t-3","precip_mm_t-4",
    "condition_t-1", "condition_t-2", "condition_t-3", "condition_t-4",
    "hour", "day_of_month", "day_of_week", "month", "quarter", "week"
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
train_data = assembler.transform(train_data).select("features", "label", "datetime")
test_data = assembler.transform(test_data).select("features", "label", "datetime")


In [15]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# Bước 1: Khởi tạo mô hình Random Forest
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Bước 2: Xác định không gian tham số cho Grid Search
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [10, 50, 100])  # Số cây trong Random Forest
             .addGrid(rf.maxDepth, [5, 10, 20])  # Chiều sâu của cây
             .build())

# Bước 3: Khởi tạo CrossValidator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# Bước 4: Tạo CrossValidator với 3-fold cross-validation
cv = CrossValidator(estimator=rf,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=3)  # Số lần cross-validation

# Bước 5: Huấn luyện mô hình với dữ liệu huấn luyện
cvModel = cv.fit(train_data)

# Bước 6: Dự đoán trên dữ liệu kiểm tra
predictions = cvModel.transform(test_data)

# Bước 7: Đánh giá mô hình
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

# Lưu mô hình đã huấn luyện (tuỳ chọn)
cvModel.save("random_forest_model")



24/12/10 15:00:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 15:00:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 15:00:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 15:00:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/12/10 15:00:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 15:00:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance

Accuracy: 0.6924603174603174


In [5]:
from pyspark.ml.tuning import CrossValidatorModel

# Đường dẫn tới thư mục lưu mô hình
model_path = "random_forest_model"

# Tải lại mô hình CrossValidatorModel
cvModel = CrossValidatorModel.load(model_path)

# Sau khi tải lại mô hình, bạn có thể truy cập mô hình Random Forest đã được huấn luyện từ CrossValidatorModel
rf_model = cvModel.bestModel

# Kiểm tra mô hình Random Forest
print(rf_model)


RandomForestClassificationModel: uid=RandomForestClassifier_2a9cc44074e7, numTrees=50, numClasses=21, numFeatures=26


In [8]:
from pyspark.sql.functions import col, lag, hour, dayofmonth, dayofweek, month, quarter, weekofyear, to_date
from pyspark.sql.window import Window

# Bước 1: Tải dữ liệu mới từ Parquet
parquet_path = "/home/hoan123/project-master/delta_lake/delta_lake_table_3/"
newdf = spark.read.parquet(parquet_path)

# Bước 2: Lấy 4 bản ghi mới nhất dựa trên cột datetime
newdf_sorted = newdf.orderBy(col("datetime").desc()).limit(10)

# Bước 3: Tạo các cột lag cho dữ liệu giống như trong dữ liệu huấn luyện
window = Window.orderBy("datetime")

condition_to_index = {
    "Cloudy": 0,
    "Light rain": 1,
    "Patchy rain possible": 2,
    "Moderate or heavy rain shower": 3,
    "Patchy light rain": 4,
    "Clear": 5,
    "Partly cloudy": 6,
    "Sunny": 7,
    "Patchy light drizzle": 8,
    "Light drizzle": 9,
    "Patchy light rain with thunder": 10,
    "Mist": 11,
    "Moderate rain at times": 12,
    "Thundery outbreaks possible": 13,
    "Moderate or heavy rain with thunder": 14,
    "Overcast": 15,
    "Moderate rain": 16,
    "Light rain shower": 17,
    "Heavy rain": 18,
    "Heavy rain at times": 19,
    "Fog": 20
}

# Bước 2: Hàm chuyển đổi chuỗi sang chỉ số
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def map_condition(condition):
    return condition_to_index.get(condition, -1)  # Trả về -1 nếu không tìm thấy giá trị phù hợp

map_condition_udf = udf(map_condition, IntegerType())
newdf_sorted = newdf_sorted.withColumn("condition_index", map_condition_udf("condition"))

# Hiển thị kết quả
newdf_sorted.show()


+-------------------+-----+-------+--------+---------+------+--------+--------+---------+-------------+---------------+
|           datetime| name|country|latitude|longitude|temp_c|wind_mph|humidity|precip_mm|    condition|condition_index|
+-------------------+-----+-------+--------+---------+------+--------+--------+---------+-------------+---------------+
|2024-11-19 23:00:00|Hanoi|Vietnam| 21.0333|   105.85|  23.1|     3.8|    69.0|      0.0|Partly cloudy|              6|
|2024-11-19 22:00:00|Hanoi|Vietnam| 21.0333|   105.85|  23.8|     4.3|    66.0|      0.0|        Clear|              5|
|2024-11-19 21:00:00|Hanoi|Vietnam| 21.0333|   105.85|  24.5|     4.9|    62.0|      0.0|Partly cloudy|              6|
|2024-11-19 20:00:00|Hanoi|Vietnam| 21.0333|   105.85|  25.0|     4.5|    59.0|      0.0|Partly cloudy|              6|
|2024-11-19 19:00:00|Hanoi|Vietnam| 21.0333|   105.85|  25.8|     1.8|    55.0|      0.0|       Cloudy|              0|
|2024-11-19 18:00:00|Hanoi|Vietnam| 21.0

In [9]:

newdf_with_lags = (
    newdf_sorted
    .withColumn("temp_c_t-1", lag("temp_c", 1).over(window))
    .withColumn("temp_c_t-2", lag("temp_c", 2).over(window))
    .withColumn("temp_c_t-3", lag("temp_c", 3).over(window))
    .withColumn("temp_c_t-4", lag("temp_c", 4).over(window))
    .withColumn("humidity_t-1", lag("humidity", 1).over(window))
    .withColumn("humidity_t-2", lag("humidity", 2).over(window))
    .withColumn("humidity_t-3", lag("humidity", 3).over(window))
    .withColumn("humidity_t-4", lag("humidity", 4).over(window))
    .withColumn("wind_mph_t-1", lag("wind_mph", 1).over(window))
    .withColumn("wind_mph_t-2", lag("wind_mph", 2).over(window))
    .withColumn("wind_mph_t-3", lag("wind_mph", 3).over(window))
    .withColumn("wind_mph_t-4", lag("wind_mph", 4).over(window))
    .withColumn("precip_mm_t-1", lag("precip_mm", 1).over(window))
    .withColumn("precip_mm_t-2", lag("precip_mm", 2).over(window))
    .withColumn("precip_mm_t-3", lag("precip_mm", 3).over(window))
    .withColumn("precip_mm_t-4", lag("precip_mm", 4).over(window))
    .withColumn("condition_t-1", lag("condition_index", 1).over(window))
    .withColumn("condition_t-2", lag("condition_index", 2).over(window))
    .withColumn("condition_t-3", lag("condition_index", 3).over(window))
    .withColumn("condition_t-4", lag("condition_index", 4).over(window))
    .withColumn("hour", hour("datetime"))
    .withColumn("day_of_month", dayofmonth("datetime"))
    .withColumn("day_of_week", dayofweek("datetime"))
    .withColumn("month", month("datetime"))
    .withColumn("quarter", quarter("datetime"))
    .withColumn("week", weekofyear("datetime"))
)

# Bước 4: Chọn các cột cần thiết và chuyển đổi tên cột như dữ liệu huấn luyện
newdf_final = newdf_with_lags.select(
    "datetime", 
    "temp_c_t-1", "temp_c_t-2", "temp_c_t-3", "temp_c_t-4", 
    "humidity_t-1", "humidity_t-2", "humidity_t-3", "humidity_t-4",
    "wind_mph_t-1", "wind_mph_t-2","wind_mph_t-3","wind_mph_t-4",
    "precip_mm_t-1", "precip_mm_t-2","precip_mm_t-3","precip_mm_t-4",
    "condition_t-1", "condition_t-2", "condition_t-3", "condition_t-4",  
    "hour", "day_of_month", "day_of_week", "month", "quarter", "week",
    "condition_index"  # Nhiệt độ hiện tại làm nhãn
).withColumnRenamed("condition_index", "label")

# Bước 5: Bỏ các hàng không đủ dữ liệu (nếu có)
newdf_final = newdf_final.na.drop()

# Bước 6: Chuyển dữ liệu thành feature vector
feature_cols = [
    "temp_c_t-1", "temp_c_t-2", "temp_c_t-3", "temp_c_t-4", 
    "humidity_t-1", "humidity_t-2",  "humidity_t-3", "humidity_t-4",
    "wind_mph_t-1", "wind_mph_t-2", "wind_mph_t-3","wind_mph_t-4",
    "precip_mm_t-1", "precip_mm_t-2","precip_mm_t-3","precip_mm_t-4",
    "condition_t-1", "condition_t-2", "condition_t-3", "condition_t-4",
    "hour", "day_of_month", "day_of_week", "month", "quarter", "week"
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
newdf_final = assembler.transform(newdf_final).select("features", "label", "datetime")

# Kết quả: DataFrame đã chuẩn bị để dự đoán
newdf_final.show()



24/12/10 16:19:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 1

+--------------------+-----+-------------------+
|            features|label|           datetime|
+--------------------+-----+-------------------+
|[27.4,28.7,28.8,2...|    5|2024-11-19 18:00:00|
|[26.4,27.4,28.7,2...|    0|2024-11-19 19:00:00|
|[25.8,26.4,27.4,2...|    6|2024-11-19 20:00:00|
|[25.0,25.8,26.4,2...|    6|2024-11-19 21:00:00|
|[24.5,25.0,25.8,2...|    5|2024-11-19 22:00:00|
|[23.8,24.5,25.0,2...|    6|2024-11-19 23:00:00|
+--------------------+-----+-------------------+



In [10]:
# Giả sử bạn đã huấn luyện mô hình RandomForest
# Dự đoán trên dữ liệu đã chuẩn bị
predictions = rf_model.transform(newdf_final)

# Lấy các dự đoán cho 4 bản ghi cuối
predictions.show()

# Nếu bạn muốn chỉ lấy các dự đoán:
predictions.select("datetime", "features", "label", "prediction").show()


24/12/10 16:19:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 1

+--------------------+-----+-------------------+--------------------+--------------------+----------+
|            features|label|           datetime|       rawPrediction|         probability|prediction|
+--------------------+-----+-------------------+--------------------+--------------------+----------+
|[27.4,28.7,28.8,2...|    5|2024-11-19 18:00:00|[0.26354412930457...|[0.00527088258609...|       7.0|
|[26.4,27.4,28.7,2...|    0|2024-11-19 19:00:00|[0.15047413417776...|[0.00300948268355...|       5.0|
|[25.8,26.4,27.4,2...|    6|2024-11-19 20:00:00|[8.37618595841209...|[0.16752371916824...|       5.0|
|[25.0,25.8,26.4,2...|    6|2024-11-19 21:00:00|[2.85981272330931...|[0.05719625446618...|       6.0|
|[24.5,25.0,25.8,2...|    5|2024-11-19 22:00:00|[2.57874450283250...|[0.05157489005665...|       6.0|
|[23.8,24.5,25.0,2...|    6|2024-11-19 23:00:00|[1.95715963015061...|[0.03914319260301...|       5.0|
+--------------------+-----+-------------------+--------------------+-------------

24/12/10 16:19:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 16:19:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/10 1

+-------------------+--------------------+-----+----------+
|           datetime|            features|label|prediction|
+-------------------+--------------------+-----+----------+
|2024-11-19 18:00:00|[27.4,28.7,28.8,2...|    5|       7.0|
|2024-11-19 19:00:00|[26.4,27.4,28.7,2...|    0|       5.0|
|2024-11-19 20:00:00|[25.8,26.4,27.4,2...|    6|       5.0|
|2024-11-19 21:00:00|[25.0,25.8,26.4,2...|    6|       6.0|
|2024-11-19 22:00:00|[24.5,25.0,25.8,2...|    5|       6.0|
|2024-11-19 23:00:00|[23.8,24.5,25.0,2...|    6|       5.0|
+-------------------+--------------------+-----+----------+



24/12/10 16:19:55 WARN DAGScheduler: Broadcasting large task binary with size 9.7 MiB


In [11]:
# Dự đoán bước 1
pred_1 = rf_model.transform(newdf_final)
df_pred_1 = pred_1.withColumn("predicted_label_t+1", col("prediction"))

# Cập nhật dữ liệu cho bước 2 (dùng pred_1 làm đầu vào cho bước tiếp theo)
df_pred_2 = (
    df_pred_1
    .withColumn("temp_c_t-1", col("temp_c_t-1").alias("temp_c_t-2"))
    .withColumn("temp_c_t-2", col("temp_c_t-2").alias("temp_c_t-3"))
    .withColumn("temp_c_t-3", col("temp_c_t-3").alias("temp_c_t-4"))
    .withColumn("temp_c_t-4", col("predicted_label_t+1"))  # Sử dụng dự đoán làm giá trị mới cho t-4
    .withColumn("condition_t-1", col("condition_t-1").alias("condition_t-2"))
    .withColumn("condition_t-2", col("condition_t-2").alias("condition_t-3"))
    .withColumn("condition_t-3", col("condition_t-3").alias("condition_t-4"))
    .withColumn("condition_t-4", col("predicted_label_t+1"))  # Cập nhật giá trị của condition
)

# Dự đoán bước 2
pred_2 = rf_model.transform(df_pred_2)
df_pred_2 = pred_2.withColumn("predicted_label_t+2", col("prediction"))

# Cập nhật dữ liệu cho bước 3
df_pred_3 = (
    df_pred_2
    .withColumn("temp_c_t-1", col("temp_c_t-1").alias("temp_c_t-2"))
    .withColumn("temp_c_t-2", col("temp_c_t-2").alias("temp_c_t-3"))
    .withColumn("temp_c_t-3", col("temp_c_t-3").alias("temp_c_t-4"))
    .withColumn("temp_c_t-4", col("predicted_label_t+2"))  # Sử dụng dự đoán làm giá trị mới cho t-4
    .withColumn("condition_t-1", col("condition_t-1").alias("condition_t-2"))
    .withColumn("condition_t-2", col("condition_t-2").alias("condition_t-3"))
    .withColumn("condition_t-3", col("condition_t-3").alias("condition_t-4"))
    .withColumn("condition_t-4", col("predicted_label_t+2"))  # Cập nhật giá trị của condition
)

# Dự đoán bước 3
pred_3 = rf_model.transform(df_pred_3)

# Hiển thị kết quả
pred_3.select("datetime", "features", "label", "prediction", "predicted_label_t+1", "predicted_label_t+2", "prediction").show()


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `temp_c_t-1` cannot be resolved. Did you mean one of the following? [`features`, `prediction`, `datetime`, `label`, `probability`].;
'Project [features#1754, label#1686, datetime#915, rawPrediction#1889, probability#1897, prediction#1909, predicted_label_t+1#1925, 'temp_c_t-1 AS temp_c_t-1#1934]
+- Project [features#1754, label#1686, datetime#915, rawPrediction#1889, probability#1897, prediction#1909, prediction#1909 AS predicted_label_t+1#1925]
   +- Project [features#1754, label#1686, datetime#915, rawPrediction#1889, probability#1897, UDF(rawPrediction#1889) AS prediction#1909]
      +- Project [features#1754, label#1686, datetime#915, rawPrediction#1889, UDF(rawPrediction#1889) AS probability#1897]
         +- Project [features#1754, label#1686, datetime#915, UDF(features#1754) AS rawPrediction#1889]
            +- Project [features#1754, label#1686, datetime#915]
               +- Proje